In [2]:
# Hide output of this cell
%%capture

# Install packages
!pip install fastcore fastai --upgrade
!pip install rasterio
!pip install geopandas

# Import packages
from fastai.vision.all import *
import rasterio.features
import shapely.geometry
import pandas as pd
import geopandas as gpd
import fiona
import numpy as np
import subprocess as sp
import natsort 
import random
import os
from matplotlib import pyplot as plt
import glob
import pytz
from datetime import datetime
from google.colab import drive

In [3]:
# Check, which GPU was allocated 
!nvidia-smi

Mon Jan 24 10:31:45 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.46       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    26W / 250W |      2MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [7]:
# Set type of imagery, mask, and  to train model with
imagery_type = "aerial" # or "sattelite"
mask_type = "buildings" # "settlements"
#mask_subfolder = "/Manually Labelled"
mask_subfolder = "2019_10cm_RGB_BE_67"
tile_type = "/512_512 stride" # smaller images: replace "512_512" with "256_256"
if "512" in tile_type:
  tile_size = "512"
elif "256" in tile_type:
  tile_size = "256"

# Set path of root folder of images and masks
path = Path(f"/content/drive/MyDrive/Segmentation Data/{imagery_type}/") # adapt to your gdrive folder structure

# Mount Google Drive for Data Access
drive.mount('/content/drive')

# Get all image and label tiles and check if their number is equal
fnames = get_image_files(path/f'image_tiles/{mask_subfolder}{tile_type}')
lbl_names = get_image_files(path/f'{mask_type}_mask_tiles/{mask_subfolder}/{tile_type}')

# Check number of image and mask tiles
if len(fnames) == len(lbl_names):
  print("Number of both image and mask tiles : ", len(lbl_names))
elif len(fnames) != len(lbl_names):
  print("ERROR: unequal number of image and mask tiles!")

# Set codes and label functions
if mask_type == "settlements":
  codes = ['Formal Area', 'Informal Settlement']

elif mask_type == "buildings":
  codes = ['Background', 'Building']

# Get codes and masks for files
def n_codes(fnames, is_partial = True):
  "Gather the codes from a list of `fnames`"
  vals = set()
  if is_partial:
    random.shuffle(fnames)
    fnames = fnames[:10]
  for fname in fnames:
    msk = np.array(PILMask.create(fname))
    for val in np.unique(msk):
      if val not in vals:
        vals.add(val)
  vals = list(vals)
  p2c = dict()
  for i,val in enumerate(vals):
    p2c[i] = vals[i]
  return p2c

def get_msk(fn, pix2class):
  "Grab a mask from a `filename` and adjust the pixels based on `pix2class`"
  if imagery_type == "satellite":
    fn = path/f'{mask_type}_mask_tiles/{mask_subfolder}{tile_type}'/f'{fn.stem}{fn.suffix}'
  if imagery_type == "aerial":
    fn = path/f'{mask_type}_mask_tiles/{mask_subfolder}{tile_type}'/f'{fn.stem[:-3]}lbl{fn.suffix}'
  msk = np.array(PILMask.create(fn))
  mx = np.max(msk)
  for i, val in enumerate(p2c):
    msk[msk==p2c[i]] = val
  return PILMask.create(msk)


# Convert mask values 255 to 1 and use function in dataloader
p2c = n_codes(lbl_names)
get_y = lambda o: get_msk(o, p2c)

# Function to create missing folders
def create_missing_folder(folder):
  if not os.path.exists(folder):
    os.makedirs(folder)

# Function to automatically set batch size depending on image size and architecture used (with .to_fp16())
def batch_size(architecture, tile_size):
  if "512" in tile_size:
    batch_size_dict = {"resnet152": 2, "resnet101": 2, "resnet50": 2,
                       "resnet34": 11, "resnet18": 10}
  elif "256" in tile_size:
    batch_size_dict = {"resnet152": 2, "resnet101": 2, "resnet50": 2,
                       "resnet34": 11, "resnet18": 10}
  bs = batch_size_dict[architecture]
  return bs

def timestamp():
  tz = pytz.timezone('Europe/Berlin')
  date = str(datetime.now(tz)).split(" ")
  date_time = f"{date[0]}_{date[1].split('.')[0][:5]}"
  return date_time

# Combining several arrays back into one (to restitch the image tiles)
def unblockshaped(arr, h, w):
    """
    Return an array of shape (h, w) where
    h * w = arr.size

    If arr is of shape (n, nrows, ncols), n sublocks of shape (nrows, ncols),
    then the returned array preserves the "physical" layout of the sublocks.

    Source: https://stackoverflow.com/a/16873755
    """
    try: # with color channel
      n, nrows, ncols, c = arr.shape
      return (arr.reshape(h//nrows, -1, nrows, ncols, c)
                .swapaxes(1,2)
                .reshape(h, w, c))
    except ValueError: # without color channel
      n, nrows, ncols = arr.shape
      return (arr.reshape(h//nrows, -1, nrows, ncols)
                .swapaxes(1,2)
                .reshape(h, w))

# Predict images, stitch predictions together, and save as shapefile      
def create_prediction_shp(input_path, input_name, output_path):
  start_time = datetime.now()
  
  test_dl = dls.test_dl(get_image_files(input_path))
  preds = learn.get_preds(dl = test_dl)
  predicted_classes = preds[0].argmax(axis = 1)
  
  end_time = datetime.now()
  print('Duration: {}'.format(end_time - start_time))

  fnames_eval = []
  num = 0

  # Get filenames
  for filepath in glob.glob(f"{input_path}/*"):
    filepath = filepath.strip(".png")
    fnames_eval.append(filepath)

  if not os.path.exists(str(output_path)):
      os.makedirs(str(output_path))

  # Write new images from prediction
  for pred in predicted_classes.numpy():
    matplotlib.image.imsave(f'{output_path}/{fnames_eval[num].split("/")[-1]}_pred.png', pred) # old: nppred
    num +=1

  # Get predicted tiles and order them 
  mask_tiles = get_image_files(path/"inference/output")
  mask_tiles = sorted(mask_tiles, key = lambda a: (int(str(a).split("/")[-1].split("_")[-3]), int(str(a).split("/")[-1].split("_")[-2])))

  # Stitch predicted tiles back together and save as PNG file
  mask_arrs = np.array(list(maps(partial(load_image, mode = "L"), np.asarray, mask_tiles)))
  mask_array = unblockshaped(np.array(mask_arrs), 10000, 10000) # h, w = (nrows//2)*n
  #im = Image.fromarray(mask_array)
  #im.save(f"{path}/inference/Prediction.png", "PNG")

  # Write predictions to shapefile
  with rasterio.open(f"{path}/images/{input_name}", "r") as src:
    raster_meta = src.meta

  shapes = rasterio.features.shapes(mask_array, transform = raster_meta["transform"])
  start_time = datetime.now()
  polygons = [shapely.geometry.Polygon(shape[0]["coordinates"][0]) for shape in shapes]
  end_time = datetime.now()
  print("Duration of creating polygons:", (end_time - start_time))
  print("No. of polygons: ", len(polygons))
  start_time = datetime.now()

  gdf = gpd.GeoDataFrame(crs = raster_meta["crs"], geometry = polygons)
  gdf["area"] = gdf['geometry'].area
  # Drop shapes that are too small or too large to be a building
  gdf = gdf[(gdf["area"] > 3) & (gdf["area"] < 500000)] # drop all detected structures smaller than 3sqm
  #gdf.to_file(f"{path}/inference/shapefiles/Prediction_{input_name}.shp", driver = 'ESRI Shapefile')
  gdf.to_file(f"Prediction_{input_name}.shp", driver = 'ESRI Shapefile')
  end_time = datetime.now()
  print('Duration: of creating shapefile: ', (end_time - start_time))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Number of both image and mask tiles :  400


UNET

In [8]:
# Create image augmentations
tfms = [*aug_transforms(mult = 1.0, do_flip = True, flip_vert = False, max_rotate = 10.0, min_zoom = 1.0, max_zoom = 1.1, max_warp = 0.2), Normalize.from_stats(*imagenet_stats)] 

# Set model, architecture, loss function, and fit function
architecture = "U-Net"
backbone = "resnet18"

loss_function = "Default Loss"
fit_type = "FOC" # fit_one_cycle: FOC, fine_tune: FT

# Automatically set batch size depending on image size and architecture used (with .to_fp16())
bs = batch_size(backbone, tile_size)

dls = SegmentationDataLoaders.from_label_func(path, fnames, label_func = get_y, bs = 2, codes = codes, seed = 2, batch_tfms = tfms)
  
# Set learner architecture and callbacks
learn = unet_learner(dls, resnet18,  n_out = 2, metrics = [Dice()]).to_fp16() # 16-bits floats, which take half the space in RAM

model_dir = f"{path}/models/Ferris" # Change name
create_missing_folder(model_dir)

# Set loggers
cbs = [CSVLogger(fname = f"{model_dir}/{architecture}_{backbone}_{loss_function}_{fit_type}_{timestamp()}.csv", append = True),
       ShowGraphCallback(),
       EarlyStoppingCallback(monitor = 'dice', patience = 10, reset_on_fit = True)]

/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:1051: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  ret = func(*args, **kwargs)


# Sentinel

In [ ]:
learn.load("/content/drive/MyDrive/Segmentation Data/sentinel/saved_model")

# Test on Sentinel
test_dl = dls.test_dl(get_image_files("/content/drive/MyDrive/Segmentation Data/sentinel/tiled"))
preds = learn.get_preds(dl = test_dl)

fnames_eval = []
num = 0

# Get filenames
for filepath in glob.glob("/content/drive/MyDrive/Segmentation Data/sentinel/tiled/*"):
  filepath = filepath.strip(".jp2")
  fnames_eval.append(filepath)

# Write new images from prediction
for pred in preds[0]:
  nppred = pred[0].numpy()
  nppred[nppred < .5] = 0
  matplotlib.image.imsave(f'{fnames_eval[num]}_pred.png', nppred)
  num +=1
  

# Aerial

Run prediction on tiles and stitch output together to one prediction mask

In [9]:
# Load saved prediction model
learn = learn.load(path/"models/Ferris/saved/U-Net_resnet18_Dual_Focal_loss_FOC_2022-01-04_18:02_saved_model")

# Set directories for tiles to predict on and where to store predictions
input_path = path/"inference/input"
input_name = "2019_10cm_RGB_BE_68.tif"
output_path = path/"inference/output"

In [ ]:
for i in range(2500):# Run (timed) prediction
  start_time = datetime.now()
  create_prediction_shp(input_path, input_name, output_path)
  end_time = datetime.now()
  print(f"Duration of prediction and creation of shapefile: {end_time - start_time}")

Duration: 0:00:35.970984
Duration of creating polygons: 0:00:02.831474
No. of polygons:  28537
Duration: of creating shapefile:  0:00:00.551247
Duration of prediction and creation of shapefile: 0:03:33.193681


Duration: 0:00:31.091409
Duration of creating polygons: 0:00:03.010286
No. of polygons:  28537
Duration: of creating shapefile:  0:00:00.539329
Duration of prediction and creation of shapefile: 0:00:51.405917


Duration: 0:00:31.050876
Duration of creating polygons: 0:00:03.045468
No. of polygons:  28537
Duration: of creating shapefile:  0:00:00.523811
Duration of prediction and creation of shapefile: 0:00:51.535950


Duration: 0:00:30.987284
Duration of creating polygons: 0:00:02.890541
No. of polygons:  28537
Duration: of creating shapefile:  0:00:00.543155
Duration of prediction and creation of shapefile: 0:00:51.229139


Duration: 0:00:30.830078
Duration of creating polygons: 0:00:03.085492
No. of polygons:  28537
Duration: of creating shapefile:  0:00:00.543421
Duration of prediction and creation of shapefile: 0:00:51.338351


...

Duration: 0:00:30.777335
Duration of creating polygons: 0:00:02.928315
No. of polygons:  28537
Duration: of creating shapefile:  0:00:00.520665
Duration of prediction and creation of shapefile: 0:00:50.588395


Duration: 0:00:30.859050
Duration of creating polygons: 0:00:02.801993
No. of polygons:  28537
Duration: of creating shapefile:  0:00:00.536091
Duration of prediction and creation of shapefile: 0:00:50.623662


Duration: 0:00:30.728092
Duration of creating polygons: 0:00:02.993589
No. of polygons:  28537
Duration: of creating shapefile:  0:00:00.525774
Duration of prediction and creation of shapefile: 0:00:50.539685


Duration: 0:00:30.576879
Duration of creating polygons: 0:00:02.942231
No. of polygons:  28537
Duration: of creating shapefile:  0:00:00.518787
Duration of prediction and creation of shapefile: 0:00:50.446299


Duration: 0:00:30.457462
Duration of creating polygons: 0:00:02.865621
No. of polygons:  28537
Duration: of creating shapefile:  0:00:00.529449
Duration of prediction and creation of shapefile: 0:00:50.158713


Sanity Check

In [ ]:
# Compare number of input tiles and predicted tiles
print("to_predict: ", len(get_image_files(input_path)))
print("mask_tiles: ", len(get_image_files(output_path)))

to_predict:  400
mask_tiles:  400


# WIP and old code

Convert to Shapefile

In [ ]:
with rasterio.open("/content/drive/MyDrive/Segmentation Data/aerial/images/2019_10cm_RGB_BE_68.tif", "r") as src:
  raster_meta = src.meta

shapes = rasterio.features.shapes(mask_array, transform = raster_meta["transform"])
start_time = datetime.now()
polygons = [shapely.geometry.Polygon(shape[0]["coordinates"][0]) for shape in shapes]
end_time = datetime.now()
print("Duration of creating polygons:", (end_time - start_time))
print("No. of polygons: ", len(polygons))
start_time = datetime.now()

gdf = gpd.GeoDataFrame(crs = raster_meta["crs"], geometry = polygons)
gdf["area"] = gdf['geometry'].area
# Drop shapes that are too small or too large to be a building
#gdf = gdf[(gdf["area"] > 2) & (gdf["area"] < 500000)]
gdf.to_file("/content/drive/MyDrive/Segmentation Data/aerial/Prediction_all.shp", driver = 'ESRI Shapefile')
end_time = datetime.now()
print('Duration: of creating shapefile: ', (end_time - start_time))

AttributeError: ignored

In [ ]:
import rasterio as rio
from rasterio.enums import Resampling
from rasterio.plot import show

#Input png image, to convert as geotiff
raster_path = "/content/drive/MyDrive/Segmentation Data/aerial/images/2019_10cm_RGB_BE_68.tif"

img = rio.open("/content/drive/MyDrive/Segmentation Data/aerial/Prediction.png")
img = img.read()
img = img.astype('uint16')
show(img) #shows true color

# Input image for coordinate reference
with rio.open(raster_path) as naip:
    #open georeferenced.tif for writing
    with rio.open(
        'georeferenced.tif',
        'w',
        driver='GTiff',
        count=img.shape[0],
        height=img.shape[1],
        width=img.shape[2],
        dtype=img.dtype,
        crs=naip.crs,
        transform=naip.transform,
        ) as dst:
            dst.write(img)

with rio.open('georeferenced.tif') as limg:
        show(limg) #1 band only shows
        show(limg.read()) #shows true color
        #resample so pixels overlap with reference
        limg = limg.read(out_shape=(1,naip.shape[0],naip.shape[1]),
                         resampling=Resampling.nearest)
        with rio.open('resampled.tif','w', 
                     driver='GTiff',
                     count=limg.shape[0],
                     height=limg.shape[1],
                     width=limg.shape[2],
                     dtype=limg.dtype,
                     crs=naip.crs,
                     transform=naip.transform,
                     ) as dst:
            dst.write(limg)

In [ ]:
from contextlib import contextmanager
import rasterio

# use context manager so DatasetReader and MemoryFile get cleaned up automatically
@contextmanager
def mem_raster(array, source_ds):
    profile = source_ds.profile
    profile.update(driver='GTiff', dtype = array.dtype)

    with rasterio.MemoryFile() as memfile:
        with memfile.open(**profile) as dataset: # Open as DatasetWriter
            dataset.write(array)

        with memfile.open() as dataset:  # Reopen as DatasetReader
            yield dataset  # Note yield not return


with rasterio.open('/content/drive/MyDrive/Segmentation Data/aerial/Prediction.png') as src:
    array = src.read() / 2.0
    with mem_raster(array, src) as mem:
        print(mem.dtypes, mem.crs, mem.bounds)
        print(repr(mem))

/usr/local/lib/python3.7/dist-packages/rasterio/__init__.py:220: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
/usr/local/lib/python3.7/dist-packages/rasterio/io.py:137: NotGeoreferencedWarning: The given matrix is equal to Affine.identity or its flipped counterpart. GDAL may ignore this matrix and save no geotransform without raising an error. This behavior is somewhat driver-specific.
  nodata=nodata, sharing=sharing, **kwargs)


('float64',) None BoundingBox(left=0.0, bottom=10000.0, right=10000.0, top=0.0)
<open DatasetReader name='/vsimem/6f1545ba-9601-43fb-aad5-6cff5dd45fc0/6f1545ba-9601-43fb-aad5-6cff5dd45fc0.tif' mode='r'>


In [ ]:
maskShape = rasterio.features.shapes(mask_array.astype('uint8'), connectivity = 8, transform = transform)
mypoly=[]
counter = 0
for vec in maskShape:
    mypoly.append(vec[0])
    counter += 1
    if counter %10:
      print(counter)
print(mypoly)

In [ ]:
import rasterio.features
from shapely.geometry import shape

# Mask is a numpy array binary mask loaded however needed
maskShape = rasterio.features.shapes(mask_array.astype('uint8'))
mypoly=[]
for vec in maskShape:
    mypoly.append(vec[0])
print(mypoly)

KeyboardInterrupt: ignored

In [ ]:
def save_predictions(scene, path=path) -> None:
  "Predicts all 16 tiles of one scene and saves them to disk"
  output_folder = path/scene/'predicted_tiles'
  if not os.path.exists(output_folder):
    os.makedirs(output_folder)
  tiles = get_image_files(path/scene/'img_tiles').sorted()
  for i in range(16):
    tile_preds = learn.predict(tiles[i])
    to_image(tile_preds[2][1].repeat(3,1,1)).save(output_folder/f'{i:02d}.png')

learn.load("/content/drive/MyDrive/Segmentation Data/aerial/models/Ferris/saved/U-Net_resnet18_Default loss_FOC default lr bs 8 valid 0.1 no tfms_2022-01-05_10:54_saved_model")

In [ ]:
save_predictions()

In [ ]:
def unblockshaped(arr, h, w):
    """
    Return an array of shape (h, w) where
    h * w = arr.size

    If arr is of shape (n, nrows, ncols), n sublocks of shape (nrows, ncols),
    then the returned array preserves the "physical" layout of the sublocks.

    Source: https://stackoverflow.com/a/16873755
    """
    try: # with color channel
      n, nrows, ncols, c = arr.shape
      return (arr.reshape(h//nrows, -1, nrows, ncols, c)
                .swapaxes(1,2)
                .reshape(h, w, c))
    except ValueError: # without color channel
      n, nrows, ncols = arr.shape
      return (arr.reshape(h//nrows, -1, nrows, ncols)
                .swapaxes(1,2)
                .reshape(h, w))
      

def get_saved_preds(path):
  "Load saved prediction mask tiles for a scene and return image + assembled mask"
  image_file = (path/'images/').ls()[1]
  image = load_image(image_file)

  mask_tiles = get_image_files(path/'eval_tiles/preds').sorted()
  mask_arrs = np.array(list(maps(partial(load_image, mode="L"), np.asarray, mask_tiles)))
  mask_array = unblockshaped(np.array(mask_arrs), 51200, 51200)

  return (image, mask_array)

def show_complete_preds(image, mask_array):
  "Source: https://github.com/CosmiQ/CosmiQ_SN7_Baseline/blob/master/notebooks/sn7_baseline.ipynb"
  figsize = (25, 16)
  fig, (ax0, ax1) = plt.subplots(1, 2, figsize=figsize)
  _ = ax0.imshow(image)
  ax0.set_xticks([])
  ax0.set_yticks([])
  ax0.set_title('Image')
  _ = ax1.imshow(mask_array, cmap='viridis')
  ax1.set_xticks([])
  ax1.set_yticks([])
  ax1.set_title('Prediction Mask')
  #plt.suptitle(scene)
  plt.tight_layout()
  plt.savefig(os.path.join(path, '_im0+mask0+dice575.png'))
  plt.show()

In [ ]:
mask_tiles = get_image_files(path/'eval_tiles/preds').sorted()
mask_arrs = np.array(list(maps(partial(load_image, mode="L"), np.asarray, mask_tiles)))
mask_array = unblockshaped(np.array(mask_arrs), 5120, 5120)